In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import tweepy
from kafka import KafkaProducer
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from tweepy import Stream
import json

# Declare a Python class (MyListener) that inherit from tweepy.StreamListener
# Then we override the on_status method that will be automatically used once
# Twitter "sends" us a new tweet.
class MyListener(StreamListener):

  def __init__(self, producer, producer_topic):
    super().__init__()
    self.producer = producer
    self.producer_topic = producer_topic

  def on_status(self, status):
    tweet = {
      'user_id': status.user.id,
      'username': status.user.name,
      'screen_name': status.user.screen_name,
      'text': status.text,
      'Tweet Created At': status.created_at,
      'User Location': status.user.location,
      'Tweet Coordinates': status.coordinates,
      'Favorited': status.favorited,
      'Retweeted': status.retweeted,
      'Phone Type': status.source,
      'Replied': status.in_reply_to_status_id_str,
      'favorite_count': status.favorite_count,
      'retweet_count': status.retweet_count,
      'hearts_count': status.user.favourites_count,
      'following_count': status.user.friends_count,
      'followers_count': status.user.followers_count,
      'hashtags': [],
    }
    if status.entities.get('hashtags') is not None:
      hashtags = status.entities.get('hashtags')
      tweet['hashtags'] = [ h.get('text') for h in hashtags ]
    self.producer.send(topic=self.producer_topic, value=tweet)
    
# Set consumer key, consumer secret, access token and access token secret
# Read them from your Twitter dev page
consumer_key = "24Jgnez2AMO39yWzvZT5rdA8m"
consumer_secret = "dcVJLFGAgg2DL9NqGZbEXdEb2nMuXJItos3jdbaaj4PMOMF1oI"
access_token = "1334438191668072448-omagasI8laVHrBjVA6GTQa4hwFuvk7"
access_token_secret = "FZjCnP074OCPVOs7qulXFVMN7bfUThEgPz1m0fqbg3vlb"

# Authenticate with the OAuth protocol
# See: https://en.wikipedia.org/wiki/OAuth
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Get the API object passing the authentication object
# See http://docs.tweepy.org/en/latest/api.html to discover the available methods
api = tweepy.API(auth)

producer = KafkaProducer(
  bootstrap_servers=["kafka:9092"],
  value_serializer=lambda v: json.dumps(v, indent=4, sort_keys=True, default=str).encode("utf-8"))

listener = MyListener(producer=producer, producer_topic="pfizertweet")
# We need an api to stream, so we reuse the api object we create before.
# Thus, we create a Stream object.
stream = Stream(auth = api.auth, listener=listener)

# A number of twitter streams are available through Tweepy.
# For more information on the capabilities and limitations of the different
# streams see Twitter Streaming API Documentation.

# In this example we will use filter to stream all tweets containing the word "Milano".
# The track parameter is an array of search terms to stream.
stream.filter(track=["pfizer","PFE"], languages=["en"])
